# Item 19: Provide Optioanl Behavior with Keyword Arguments

- Like most other programming languages, calling a function in Python allows for passing arguments by position.

In [4]:
def remainder(number, divisor):
    return number % divisor
assert remainder(20, 7) == 6

- All positional arguments to Python functions can also be passed by keyword, where the name of the argument is used in an assignment within the parentheses of a function call. The keyword arguments can be passed in any order as long as all of the required positional arguments are specified. You can mix and match keyword and positional arguments. These calls are equivalent.

In [5]:
remainder(20, 7)
remainder(20, divisor=7)
remainder(number=20, divisor=7)
remainder(divisor=7, number=20)

6

- Positional arguments must be specified before keyword arguments

In [6]:
remainder(number=20, 7)

SyntaxError: positional argument follows keyword argument (<ipython-input-6-fa871e527313>, line 1)

In [7]:
remainder(20,	number=7)

TypeError: remainder() got multiple values for argument 'number'

- The flexibility of keyword argumnets provides three significant benefits. 

- The first advantage is that keyword arguments make the function call clearer to new readers of the code. With the call reaminder(20, 7), it's not evident which argument is the number and which is  the divisor without looking at the implementation of the remainder method. In the call with keyword arguments, number=20 and divisor=7 make it immediately obvious which parameter is being used for each purpose.


- The second impact of keyword arguments is that they can have default values specified in the function definition. This allows a function to provide additional capabilities when you need them but lets you accept the default behavior most of the time. This can eliminate repetitive code and reduce noise.

- For example, say you want to compute the rate of fluid flowing into a vat. If the vat is also on a scale, then you could use the difference between two weight measurements at two different times to determine the flow rate.

In [10]:
def flow_rate(weight_diff, time_diff):
    return weight_diff / time_diff

weight_diff = 0.5
time_diff = 3
flow = flow_rate(weight_diff, time_diff)
print('%.3f kg per second' % flow)

0.167 kg per second


- In the typical case, it's useful to know the flow rate in kilograms per second. Other times, it'd be helpful to use the last sensor measurements to approximate larger time scales, like hours or days. You can provide this behavior in the same function bu adding an argument for the time period caling factor.

In [11]:
def flow_rate(weight_diff, time_diff, period):
    return (weight_diff / time_diff) * period

- The problem is that now you need to specify the period argument every time you call the function, even in the common case of flow rate per second.

In [12]:
flow_per_second = flow_rate(weight_diff, time_diff, 1)

- To make this less noisy, I can give the period argument a default value.

In [13]:
def flow_rate(weight_diff, time_diff, period=1):
    return (weight_diff / time_diff) * period

In [14]:
flow_per_second = flow_rate(weight_diff, time_diff)
flow_per_hour = flow_rate(weight_diff, time_diff, period=3600)

- This works well for simple default values

- The third reason to use keyword arguments is that they provide a powerful way to extend a function's parameters whule remaining backwards compatible with existing callers. This lets you provide addtional functionality without having to migrate a lot of code, reducing the change of introducing bugs.

- For example, say you want to extend the flow_rate function above to calculate flow rates weight units besides kilograms. You can do  this by adding a new optional parameter that provides a conversion rate to your preferred measurement units.

In [23]:
def flow_rate(weight_diff, time_diff, 
             period=1, units_per_kg=1):
    return ((weight_diff / units_per_kg) / time_diff) * period

- The default argument value for units_per_kg is 1, which makes the returned weight units remain as kilograms. This means that all existing callers will see no chagne in behavior. New callers to flow_rate can specify the new keyword argument to see the new behavior.

In [24]:
pounds_per_hour = flow_rate(weight_diff, time_diff,
                           period=3600, units_per_kg=2.2)

- The onlu problem with this approach is that optional keyword arguments like period and units_per_kg may still be specified as positional arguments.

## Things to Remember

- Function arguments can be specified by opsition or by keyword.
- Keywords make it clear what the purpose of each argument is when would be confusing with only positional arguments.
- Keyword arguments with default values make it easy to add new behaviors to a function, especially when the function has existing callers.
- Optional keyword arguments should always be passed by keyword instead of by position.

# Item 20: Use None and Docstrings to specify Dynamic Default Arguments

- Sometimes you need to use a non-static type as a keyword argument's default value. For example, say you want to print logging messages that are marked with the time of the logged event. In the default case, you want the message to include the time when the function was called. You might try the following approach, assuming the default arguments are reevaluated each time the function is called.

In [25]:
from datetime import datetime
from time import sleep


def log(message, when=datetime.now()):
    print('%s: %s' % (when, message))
    

log('Hi there')
sleep(0.1)
log('Hi again')

2018-01-11 13:08:49.346888: Hi there
2018-01-11 13:08:49.346888: Hi again


- The timestamps are the same because datetime.now is only executed a single time.

- When the function is defined. Default argument values are evaluated only once per module load, which usually happens when a program starts up. After the module containing this code is loaded, the datetime.now default argument will never be evluated again.

- The convention for achieving the desired result in Python is to provide a default value of None and to document the actual behavior in the docstring. When your code sees an argumnet value of None, you allocate the default value accordingly.

In [26]:
def log(message, when=None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))

- Now the timestamps will be different.

In [27]:
log('Hi there')
sleep(0.1)
log('Hi again')

2018-01-11 13:08:51.878099: Hi there
2018-01-11 13:08:51.980230: Hi again



- Using None for default argument values is especially important when the arguments are mutable. For example, say you want to load a value encoded as JSON data. If decoding the data fails, you want an empty dictionary to be returned by default. You might try this approach.

In [28]:
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

- The problem here is the same as the datetime.now example above. The dictionary specified for defauly will be shared by all calls to decode because default argument values are only evaluated once (at module load time). This can cause extremely surprising behavior.

In [29]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}


- You'd expect two different dictionaries, each with a single key and value. But modifying one seems to also modify the other. The culprit is that foo and bar are both equal to the default parameter. They are the same dictionary object.

In [30]:
assert foo is bar

In [31]:
def decode(data, default=None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default: value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

In [32]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}


## Things to Remember

- Defualt arguments are only evaluated once: during function definition at module load time. This can cause odd behaviors for dynamic values 
- Use None as the default value for keyword arguments that have a dynamic value. Document the actual default behavior in the function's docstring.

# Item 21: Enforce Clarity with Keyword-Only Arguments

- Passing arguments by keyword is a powerful feature of Python functions. The flexibility of keyword arugments enables you to write code that will be clear for your use cases.

- For example, say you want to divide one number by another but be very careful about special cases. Sometimes you want to ignore ZeroDivisionError exceptions and return infinity instead. Other times, you want to ignore OverflowError exceptions and return zer instead.

In [33]:
def safe_division(number, divisor, ignore_overflow,
                 ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
        

- Using this function is straightforward. This call will ignore the float overflow from division and will return zero.

In [34]:
result = safe_division(1, 10**500, True, False)
print(result)

0.0


- This call will ignore the error from dividing by zero and will return infinity.

In [36]:
result = safe_division(1, 0, False, True)
print(result)

inf


- The problem is that it's easy to confuse the position of the two Boolean arguments that control the exception-ignoring behavior. This can easily cause bugs that are hard to track down. One way to improve the readability of this code is to use keyword arguments. By default, the function can be overly cautious and can always re-raise exceptions.

In [39]:
def safe_division_b(number, divisor, 
                  ignore_overflow=False,
                 ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

- Then callers can use keyword arguments to specify which of the ignore flags they want to flip for specific operations, overriding the default behavior.

In [40]:
safe_division_b(1, 10**500, ignore_overflow=True)
safe_division_b(1, 0, ignore_zero_division=True)

inf

- The problem is, since these keyword arguments are optional behavior, there's nothing forcing callers of your functions to use keyword arguments for clarity. Even with the new definition of safe_division_b, you can still call it the old way with positional arguments.

In [41]:
safe_division_b(1, 10**500, True, False)

0.0

- With complex functions like this, it's better to require that callers are clear about their intentions. In Python 3, you can demand clarity by defining your functions with keyword-only arguments. These arguments can onlu be supplied by keyword, never by position.

- Here, I redefine the safe_division function to accept keyword-only arguments. The \* symbol in the argument list indicates the end of positional arguments and the beginning of keyword-only arguments.

In [43]:
def safe_division_c(number, divisor, *,
                  ignore_overflow=False,
                 ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

- Now, calling the function with positional arguments for the keyword arguments won't work.

In [44]:
safe_division_c(1, 10**500, True, False)

TypeError: safe_division_c() takes 2 positional arguments but 4 were given

In [45]:
safe_division_c(1, 0, ignore_zero_division=True)

try:
    safe_division_c(1, 0)
except ZeroDivisionError:
    print('zero!')

zero!


## Keyword-Only Arguments in Python 2

- Unfortunately, Python 2 doesn't have explicit syntax for specifying keyword-only arguments like Python 3. But you can achieve the same behavior of raising TypeErrors for invalid function calls by using the \*\*operator in argument lists. The \*\* operator is similar to the \* operator, except that instead of accepting a variable number of positional arguments, it accepts any number of keyword arguments, even when they're not defined.

In [ ]:
# Python 2
def pinrt_args(*args, **kwargs):
    print 'Positional:', args
    print 'Keyword:   ', kwargs

- To make safe_division take keyword-only arguments in Python 2, you have the function accept \*\* kwargs. Then you pop keyword arguments that you expect out of the kwargs dictionary, using the pop method's second argument to specify the default value when the key is missing. Finally, you make sure there are no more keyword arguments left in kwargs to prevent callers from supplying arguments that are invalid.

In [47]:
# Python 2
def safe_division_d(number, divisor, **kwargs):
    ignore_overflow = kwargs.pop('ignore_overflow', False)
    ignore_zero_div = kwargs.pop('ignore_zero_division', False)
    if kwargs:
        raise TypeError('Unexpected **kwrags: %r' % kwargs)
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_div:
            return float('inf')
        else:
            raise

- Now, you can call the function with or without keyword arguments.

In [48]:
safe_division_d(1, 10)
safe_division_d(1, 0, ignore_zero_division=True)
safe_division_d(1, 10**500, ignore_overflow=True)

0.0

In [49]:
safe_division_d(1, 0, False, True)

TypeError: safe_division_d() takes 2 positional arguments but 4 were given

In [50]:
safe_division_d(0, 0, unexpected=True)

TypeError: Unexpected **kwrags: {'unexpected': True}

## Things to Remember

- Keyword arguments make the intention of a function call more clear.
- Use keyword-only arguments to force callers to supply keyword arguments for potentially confusing functions, especially those that accept multiple Boolean flags.
- Python 3 supports explicit syntax for keyword-only arguments in functions.
- Python 2 can emulate keyword-only arguments for functions by using \*\* kwargs and manually raising TypeError excpetions.